In [21]:
import torch, torchvision
from torch.nn import Module,Sequential,Linear,Conv2d,BatchNorm2d,ReLU,MaxPool2d
from torch.utils.data import DataLoader
import pathlib
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms
import glob

In [6]:
batch_size = 100
class ConvNet(nn.Module):
    def __init__(self,num_classes=3):
        #giving default number of classes as 3
        super(ConvNet,self).__init__()
        
        #Input shape= (batch_size,3,150,150)
        # batch_size images in a batch x 3 channels(r,g,b) in an image x (150*150) pixels in an image.
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Applies 12 different filters and therefore obtains 12 different activation maps for all images in the btatch so only depth is changed
        #Shape= (batch_size,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Number of features is only fed as the batchnorm input
        #Shape= (batch_size,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (batch_size,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (batch_size,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (batch_size,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (batch_size,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (batch_size,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (batch_size,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (batch_size,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
        #Above output will be in matrix form, with shape (batch_size,32,75,75)
            
        output=output.view(-1,32*75*75)
        #C-1 inferrs values from other dimensions to ensure the final dimension is equla to the previous end multiplication result(batch_size,32,75,75)
        #batch_size entries with each entry as a single arra flattened from previous matrices . Each array length = 32*75*75
            
            
        output=self.fc(output)
            
        return output

In [15]:

preprocess = transforms.Compose(
    [
        transforms.Resize(150),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ]
)

Changes to be made in below cell :

Make a way to slice the dataset for each client
Compile everything to one single class
So, take number of data samples and batch size as an input to the model class.

In [16]:

train_path = '/Users/tarunvisvar/Downloads/Dataset/Handwriting/Handwriting-subset/Train'
test_path = '/Users/tarunvisvar/Downloads/Dataset/Handwriting/Handwriting-subset/Test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform = preprocess),
    batch_size=batch_size, shuffle=True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform = preprocess),
    batch_size=batch_size, shuffle=True
) 

In [18]:
root = pathlib.Path(train_path)
classes = [dir.name for dir in root.iterdir()]
classes.remove('.DS_Store')

In [19]:
classes

['Reversal', 'Normal', 'Corrected']

In [20]:
model = ConvNet(num_classes = 3)
optimizer = Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [21]:
#calculating the size of training and testing images

train_count=len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.png'))

In [22]:
train_count,test_count

(4000, 1040)

In [25]:
num_epochs = 10

In [26]:
best_accuracy = 0.0

for epoch in range(num_epochs):
     model.train()
     #Model will be in training mode and takes place on training dataset
     train_loss = 0.0
     train_accuracy = 0.0
     for images,labels in train_loader:
          # The loop runs for 'number of batches' times 
          optimizer.zero_grad()
          outputs = model(images) 
          # The images(in batches) are preprocessed while brought up by the trainloader itself
          # The images(in batches) are passed through various layers and predictions are made.
          # Those are the outputs and are compared with the labels
          # The output is a batch_size length vector containing predicted output for batch_size images

          loss = loss_func(outputs,labels)
          loss.backward() # backpropagation
          optimizer.step() # Updates the weights
          #print("loss.data = ",loss.data)
          
          # For each image, we must add the loss to training loss. But loss is given for a batch by the model
          # So, we take the loss for a batch and multiply it with the batch size to get the loss for each image in an approximate manner

          train_loss += loss.data*batch_size
          #print(outputs.data) #outputs will be of size 10 x 3, for 10 images in a batch and 3 predictions for each image in a batch
          _,predictions = torch.max(outputs.data,1)
          #print(predictions) #predictions will contain the indices of the highest value outputed for each image. Therefore, predictions will contain 10(batch_size) values of the indices(hence also the classes)
          train_accuracy+=int(torch.sum(predictions==labels.data))
     train_accuracy /= train_count
     train_loss /= train_count
     print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
     model.eval()
     #Modle will eb in the mode on evaluating on test dataset
     test_accuracy = 0.0
     for images,labels in test_loader:
          outputs = model(images)
          _,predictions = torch.max(outputs.data,1)
          #print(outputs.data)
          test_accuracy += int(torch.sum(predictions==labels.data))
     test_accuracy /= test_count
     print("Test accuracy =  ",str(test_accuracy))
     if test_accuracy>best_accuracy:
        torch.save(model,'best_checkpoint.model')
        best_accuracy=test_accuracy

KeyboardInterrupt: 

In [137]:
print(best_accuracy)

0.9548076923076924


In [138]:
loaded_model = torch.load('best_checkpoint.model')

In [139]:
for name,param in loaded_model.named_parameters():
    if param.requires_grad:
        print(name,param)

conv1.weight Parameter containing:
tensor([[[[-0.0798,  0.1037, -0.0650],
          [-0.1469, -0.0271, -0.0710],
          [ 0.0055, -0.1653,  0.1933]],

         [[-0.1393,  0.0436, -0.0150],
          [-0.0965, -0.0228, -0.0916],
          [-0.0106, -0.0282,  0.0208]],

         [[-0.1845,  0.0477,  0.1889],
          [ 0.0841,  0.1630,  0.1174],
          [-0.0038,  0.1136, -0.1315]]],


        [[[-0.0660, -0.0337, -0.1542],
          [-0.1738, -0.0893,  0.0488],
          [-0.0170, -0.1456, -0.0249]],

         [[ 0.0316,  0.0687,  0.1745],
          [-0.1044,  0.1226,  0.1334],
          [ 0.0947, -0.0266, -0.0758]],

         [[-0.1096, -0.0644,  0.1631],
          [-0.0897, -0.0504, -0.1465],
          [-0.2277, -0.0462, -0.1048]]],


        [[[ 0.0362, -0.0673, -0.1224],
          [ 0.1730, -0.0061, -0.1046],
          [-0.1686,  0.0575, -0.0285]],

         [[ 0.1916,  0.0630, -0.1797],
          [ 0.0075,  0.0879, -0.0115],
          [ 0.1452, -0.0021,  0.0287]],

         

In [140]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

548399

Inference:

Number of parameters in a model = around 5 and half lakhs

In [25]:

class HWRModel:
    def __init__(self,data_path,batch_size,local_data_count):
        self.batch_size = batch_size
        self.train_path = data_path + '/Train'
        self.test_path = data_path + '/Test'
        self.local_data_count = local_data_count # Amount of data that a user can choose 
    
    def preprocess(self,resize=150):
        transformer = transforms.Compose(
            [
                transforms.Resize(resize),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
            ]
        )
        return transformer   

    def get_model(self):
        model = ConvNet(num_classes = 3)
        optimizer = Adam(model.parameters(), lr=0.001)
        loss_func = nn.CrossEntropyLoss()

        return (model,optimizer,loss_func)
    
    #Add load dataset function.

    def load_dataset(self):
        train_loader = DataLoader(
    torchvision.datasets.ImageFolder(self.train_path,transform = self.preprocess()),
    batch_size=batch_size, shuffle=True)

        test_loader = DataLoader(
    torchvision.datasets.ImageFolder(self.test_path,transform = self.preprocess()),
    batch_size=batch_size, shuffle=True) 

        return(train_loader,test_loader)
        
    def train(self,num_epochs=10):
        model,optimizer,loss_func = self.get_model()
        best_accuracy = 0.0
        train_loader,test_loader = self.load_dataset()
        train_count=len(glob.glob(self.train_path+'/**/*.png'))
        test_count=len(glob.glob(self.test_path+'/**/*.png'))

        
        for epoch in range(num_epochs):
            model.train()
            #Model will be in training mode and takes place on training dataset
            train_loss = 0.0
            train_accuracy = 0.0
            for images,labels in train_loader:
                # The loop runs for 'number of batches' times 
                optimizer.zero_grad()
                outputs = model(images) 
                # The images(in batches) are preprocessed while brought up by the trainloader itself
                # The images(in batches) are passed through various layers and predictions are made.
                # Those are the outputs and are compared with the labels
                # The output is a batch_size length vector containing predicted output for batch_size images

                loss = loss_func(outputs,labels)
                loss.backward() # backpropagation
                optimizer.step() # Updates the weights
                #print("loss.data = ",loss.data)
                
                # For each image, we must add the loss to training loss. But loss is given for a batch by the model
                # So, we take the loss for a batch and multiply it with the batch size to get the loss for each image in an approximate manner

                train_loss += loss.data*batch_size
                #print(outputs.data) #outputs will be of size 10 x 3, for 10 images in a batch and 3 predictions for each image in a batch
                _,predictions = torch.max(outputs.data,1)
                #print(predictions) #predictions will contain the indices of the highest value outputed for each image. Therefore, predictions will contain 10(batch_size) values of the indices(hence also the classes)
                train_accuracy+=int(torch.sum(predictions==labels.data))
            train_accuracy /= train_count
            train_loss /= train_count
            print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
            model.eval()
            #Modle will eb in the mode on evaluating on test dataset
            test_accuracy = 0.0
            for images,labels in test_loader:
                outputs = model(images)
                _,predictions = torch.max(outputs.data,1)
                #print(outputs.data)
                test_accuracy += int(torch.sum(predictions==labels.data))
            test_accuracy /= test_count
            print("Test accuracy =  ",str(test_accuracy))
            if test_accuracy>best_accuracy:
                torch.save(model,'best_checkpoint.model')
                best_accuracy=test_accuracy
            
    def get_parameters(self):
        loaded_model = torch.load('best_checkpoint.model')
        params = dict()
        for name,parameters in loaded_model.named_parameters():
            params[name] = parameters
        return params
        



In [26]:
data_path = '/Users/tarunvisvar/Downloads/Dataset/Handwriting/Handwriting-subset'
batch_size = 100
local_data_count = 1000
mymodel = HWRModel(data_path,batch_size,local_data_count)

In [31]:
mymodel.train(num_epochs = 10)

Epoch: 0 Train Loss: tensor(10.6635) Train Accuracy: 0.51725
Test accuracy =   0.24423076923076922
Epoch: 1 Train Loss: tensor(1.1727) Train Accuracy: 0.7435
Test accuracy =   0.8153846153846154
Epoch: 2 Train Loss: tensor(0.6347) Train Accuracy: 0.823
Test accuracy =   0.7615384615384615
Epoch: 3 Train Loss: tensor(0.5071) Train Accuracy: 0.84175
Test accuracy =   0.9538461538461539
Epoch: 4 Train Loss: tensor(0.4548) Train Accuracy: 0.85975
Test accuracy =   0.9644230769230769
Epoch: 5 Train Loss: tensor(0.3992) Train Accuracy: 0.873
Test accuracy =   0.9375
Epoch: 6 Train Loss: tensor(0.5683) Train Accuracy: 0.85475
Test accuracy =   0.8480769230769231
Epoch: 7 Train Loss: tensor(0.2934) Train Accuracy: 0.90375
Test accuracy =   0.8692307692307693
Epoch: 8 Train Loss: tensor(0.4118) Train Accuracy: 0.8815
Test accuracy =   0.7817307692307692
Epoch: 9 Train Loss: tensor(0.2989) Train Accuracy: 0.9005
Test accuracy =   0.9759615384615384


In [34]:
mymodel.get_parameters()

{'conv1.weight': Parameter containing:
 tensor([[[[ 0.0341, -0.1215,  0.0108],
           [-0.0100, -0.1124, -0.1251],
           [-0.0024, -0.1855,  0.0289]],
 
          [[ 0.0772, -0.1251,  0.1007],
           [ 0.0745, -0.1237,  0.1196],
           [-0.0823,  0.0489, -0.0373]],
 
          [[-0.1249,  0.0107, -0.1002],
           [-0.0348, -0.0368, -0.1515],
           [-0.0325, -0.1573,  0.1918]]],
 
 
         [[[ 0.1690, -0.1520,  0.0385],
           [-0.0491,  0.0280,  0.0843],
           [ 0.1760,  0.1190,  0.0933]],
 
          [[ 0.0328,  0.1423,  0.0173],
           [-0.0836, -0.1788,  0.1195],
           [-0.0968, -0.1616, -0.1491]],
 
          [[-0.1856,  0.0442, -0.1287],
           [-0.1686, -0.1574, -0.0446],
           [-0.0516, -0.0112,  0.0941]]],
 
 
         [[[ 0.0022,  0.0936,  0.1511],
           [ 0.0622, -0.0815,  0.0032],
           [ 0.1183, -0.0808, -0.0458]],
 
          [[-0.1068,  0.1701, -0.1422],
           [ 0.0849, -0.1182, -0.0911],
           [ 0